In [1]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [ ]:
from data_retrieval import lipade_groundtruth
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from lavis.models import load_model_and_preprocess
import numpy as np
import torch

device = "cpu" # torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [3]:
blip_model, blip_vis_processors, blip_text_processors = load_model_and_preprocess(name="blip_feature_extractor", model_type="base", is_eval=True, device=device)
blip2_model, blip2_vis_processors, blip2_text_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device)

/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/lavis/models/blip_models/blip.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they ar

OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 7.50 MiB is free. Process 22304 has 3.76 GiB memory in use. Including non-PyTorch memory, this process has 2.01 GiB memory in use. Of the allocated memory 1.75 GiB is allocated by PyTorch, and 150.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
corpus = "lipade_groundtruth"
rawPath = "../results/raw/" + corpus + "/"
distancePath = "../results/distance/" + corpus + "/"

In [6]:
x,_,y = lipade_groundtruth.getDataset(mode="similar")
for i in range(len(x)):
    x[i] = Image.open(x[i]).convert('RGB')

In [34]:
blip_images_embeddings = []
blip2_images_embeddings = []

for image in tqdm(x):
    blip_sample = {"image": blip_vis_processors["eval"](image).unsqueeze(0).to(device)}
    blip2_sample = {"image": blip2_vis_processors["eval"](image).unsqueeze(0).to(device)}

    blip_images_embeddings.append(blip_model.extract_features(blip_sample, mode="image").image_embeds[:,0,:].cpu().data.numpy())
    blip2_images_embeddings.append(blip2_model.extract_features(blip2_sample, mode="image").image_embeds[:,0,:].cpu().data.numpy())

  0%|          | 0/279 [00:00<?, ?it/s]/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/lavis/models/blip2_models/blip2.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast(dtype=dtype)
100%|██████████| 279/279 [00:57<00:00,  4.82it/s]


In [37]:
blip_images_embeddings = np.array(blip_images_embeddings).reshape((len(x), -1))
blip2_images_embeddings = np.array(blip2_images_embeddings).reshape((len(x), -1))

In [39]:
blip_distance = 1 - (cosine_similarity(blip_images_embeddings, blip_images_embeddings)+1)/2
blip_distance -= np.diag(blip_distance)

blip2_distance = 1 - (cosine_similarity(blip2_images_embeddings, blip2_images_embeddings)+1)/2
blip2_distance -= np.diag(blip2_distance)

In [40]:
np.save(rawPath + "blip.npy", blip_images_embeddings)
np.save(distancePath + "blip.npy", blip_distance)

np.save(rawPath + "blip2.npy", blip2_images_embeddings)
np.save(distancePath + "blip2.npy", blip2_distance)